In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
df1 = pd.read_csv('finnum/train.csv')
x = df1.index
df2 = pd.read_csv('finnum/dev.csv')
df = pd.concat([df1,df2], ignore_index = True)
df['cat_num'] = df['category'].astype('category').cat.codes
labels = pd.get_dummies(df['cat_num'][x])
labels2 = pd.get_dummies(df['cat_num'][len(x):])

In [3]:
trainInp = np.load('trainChar.npy')
valInp = np.load('valChar.npy')
embed = np.load('embedChar.npy')
embed = embed.astype(np.float32)

In [4]:
tf.reset_default_graph()

In [5]:
learning_rate = 0.001
batch_size = 16
embedding_size = 8
filter_size = 3
filter_width = 1
stride_height = 2
conv_depth = {9: [2, 2, 2, 2],  
                   17: [4, 4, 4, 4],  
                    29: [10, 10, 4, 4]}
num_filters = [64, 128, 256, 512]
hidden1_size = 1024
hidden2_size = 512
out_size = labels.shape[1]
depth = 9

In [6]:
def main_conv(x, kernel, stride, filters_out):

    filters_in = x.get_shape()[-1]
    shape = [kernel[0], kernel[1], int(filters_in), filters_out]
    W = tf.Variable(tf.truncated_normal(shape, stddev=0.1), name='W')
    b = tf.Variable(tf.constant(0.1, shape=[filters_out]), name = 'b')


    return b,tf.nn.conv2d(x, tf.cast(W,tf.float32), [1, stride, 1, 1], padding='SAME')

In [7]:
def conv_set(input_layer, layer_set, conv_id, num_filters):
    num_filters = num_filters[layer_set]
    b, conv = main_conv(x=input_layer, kernel=(filter_size, filter_width), stride=stride_height, 
                        filters_out=num_filters)
    conv = tf.layers.batch_normalization(inputs = conv)
    conv = tf.nn.relu(tf.nn.bias_add(conv, tf.cast(b,tf.float32)), name="relu")
    return conv

In [8]:
train = (trainInp, labels)
val = (valInp, labels2)

# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

# create validation Dataset and batch it
val_data = tf.data.Dataset.from_tensor_slices(val)
val_data = val_data.shuffle(10000) # if you want to shuffle your data
val_data = val_data.batch(batch_size)

# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
txt, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
val_init = iterator.make_initializer(val_data)

In [9]:
embedding = tf.nn.embedding_lookup(embed, txt, partition_strategy='mod', name=None)
embedded_chars_expanded = tf.expand_dims(embedding, -1)

Initial convolution

In [10]:
filter_shape = [filter_size, embedding_size, 1, num_filters]
b, conv = main_conv(
    embedded_chars_expanded,
    kernel = (filter_size, embedding_size),
    stride = stride_height, 
    filters_out = num_filters[0])
out = tf.nn.relu(tf.nn.bias_add(conv, tf.cast(b,tf.float32)), name="relu")

In [11]:
out

<tf.Tensor 'relu:0' shape=(?, 274, 8, 64) dtype=float32>

In [12]:
layer_set = 0
for conv_id in range(conv_depth[depth][layer_set]):
    out = conv_set(out, layer_set, conv_id, num_filters)

In [13]:
out

<tf.Tensor 'relu_2:0' shape=(?, 69, 8, 64) dtype=float32>

In [14]:
layer_set = 1
for conv_id in range(conv_depth[depth][layer_set]):
    out = conv_set(out, layer_set, conv_id, num_filters)

In [15]:
out

<tf.Tensor 'relu_4:0' shape=(?, 18, 8, 128) dtype=float32>

In [16]:
layer_set = 2
for conv_id in range(conv_depth[depth][layer_set]):
    out = conv_set(out, layer_set, conv_id, num_filters)

In [17]:
out

<tf.Tensor 'relu_6:0' shape=(?, 5, 8, 256) dtype=float32>

In [18]:
layer_set = 3
for conv_id in range(conv_depth[depth][layer_set]):
    out = conv_set(out, layer_set, conv_id, num_filters)

In [19]:
out

<tf.Tensor 'relu_8:0' shape=(?, 2, 8, 512) dtype=float32>

In [30]:
max_pool = tf.nn.max_pool(out,
                      ksize=[1, 2, 1, 1],
                      strides=[1, 1, 1, 1],
                      padding='VALID')

In [31]:
max_pool

<tf.Tensor 'MaxPool_1:0' shape=(?, 1, 8, 512) dtype=float32>

In [32]:
multiplier = max_pool.get_shape()[2]
flatten = tf.reshape(max_pool,
                     [-1, multiplier * num_filters[layer_set]])

In [33]:
flatten

<tf.Tensor 'Reshape_1:0' shape=(?, 4096) dtype=float32>

In [34]:
conn = tf.layers.dense(flatten, hidden1_size, activation = 'relu')
conn2 = tf.layers.dense(conn, hidden2_size, activation = 'relu')

In [35]:
logits = tf.layers.dense(conn2, out_size)

In [36]:
logits

<tf.Tensor 'dense_5/BiasAdd:0' shape=(?, 7) dtype=float32>

In [37]:
n_epochs = 10
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = label, logits = logits)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

In [38]:
preds = tf.nn.softmax(conn2)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

In [39]:
with tf.Session() as sess:
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())
    # train the model n_epochs times

    for i in range(n_epochs): 
        
        sess.run(train_init)# drawing samples from train_data
        total_loss = 0
        total_right = 0
        n_batches = 0
        totalright = 0
        totalvalright = 0
        nvalbatches = 0
        totalvalloss = 0
        try:
            while True:
                #summary,acc,_, l = sess.run([summary_op,accuracy,optimizer, loss]) #use with scalar summary
                acc,_, l = sess.run([accuracy, optimizer, loss])                
                total_loss += l
                total_right += acc
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        sess.run(val_init)
        try:
            while True:
                accval,valloss = sess.run([accuracy, loss])
                totalvalright += accval
                nvalbatches += 1
                totalvalloss += valloss
        except tf.errors.OutOfRangeError:
            pass

        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        print('Accuracy {0}: {1}'.format(i, total_right/n_batches))    
        print('Average val loss epoch {0}: {1}'.format(i, totalvalloss/nvalbatches))
        print('Val_Accuracy {0}: {1}'.format(i, totalvalright/nvalbatches)) 
    prediction = sess.run(preds, feed_dict={txt: valInp})
    prediction = np.asarray(prediction)


Average loss epoch 0: 1.7967361765248435
Accuracy 0: 0.0
Average val loss epoch 0: 1.535400446425093
Val_Accuracy 0: 0.0
Average loss epoch 1: 1.4986085471652804
Accuracy 1: 0.0
Average val loss epoch 1: 1.5144715841780318
Val_Accuracy 1: 0.0
Average loss epoch 2: 1.5029488143466767
Accuracy 2: 0.0
Average val loss epoch 2: 1.502631922985645
Val_Accuracy 2: 0.0
Average loss epoch 3: 1.4950281114805313
Accuracy 3: 0.0
Average val loss epoch 3: 1.5056226481782629
Val_Accuracy 3: 0.0
Average loss epoch 4: 1.4908093734866097
Accuracy 4: 0.00014880952380952382
Average val loss epoch 4: 1.49986122009602
Val_Accuracy 4: 0.0
Average loss epoch 5: 1.4896541839554196
Accuracy 5: 0.00029761904761904765
Average val loss epoch 5: 1.5141124953614904
Val_Accuracy 5: 0.0


KeyboardInterrupt: 

In [ ]:
np.mean(np.equal(np.argmax(prediction, 1), labels2.idxmax(axis = 1)))